<a href="https://colab.research.google.com/github/Guylord/APHRC_DSE_Insprire_Hackathon_2024/blob/main/APHRC_DSE_Insprire_Hackathon_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
hdss_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_hdss_v3.csv')
facility_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_facility_v3.csv')

print('HDSS INFO')
hdss_df.info()
print('\n\nFACILITY INFO')
facility_df.info()

HDSS INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   recnr       4115 non-null   int64  
 1   firstname   4115 non-null   object 
 2   lastname    4031 non-null   object 
 3   petname     2894 non-null   object 
 4   dob         4115 non-null   object 
 5   sex         4115 non-null   int64  
 6   nationalid  0 non-null      float64
 7   hdssid      4115 non-null   object 
 8   hdsshhid    4115 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 289.5+ KB


FACILITY INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   recnr       2902 non-null   int64 
 1   firstname   2902 non-null   object
 2   lastname    2835 non-null   object
 3   petname     2511 non-null   object
 4   do

In [34]:
facility_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate,fullname
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018,Fatuma
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022,Gloria Rashida
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023,Ali Hakram
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019,Nakalema
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020,Asuman Sempa


In [5]:
for column in facility_df.select_dtypes('object').columns:
  facility_df[column] = facility_df[column].str.strip()

In [33]:
facility_df['fullname'] = facility_df['firstname'].fillna('') + ' ' + facility_df['lastname'].fillna('')

In [30]:
facility_df.duplicated(subset='fullname').sum()

694

In [22]:
facility_df['sex'].nunique()

2

In [36]:
hdss_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001,Zaina Hanifa
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002,Godfrey Maganda
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003,Kasim Ngobi
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004,Esther
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005,Sumaya Swabula


In [15]:
for column in hdss_df.select_dtypes('object').columns:
  hdss_df[column] = hdss_df[column].str.strip()

In [35]:
hdss_df['fullname'] = hdss_df['firstname'].fillna('') + ' ' + hdss_df['lastname'].fillna('')

In [26]:
hdss_df.duplicated().sum()

0

In [17]:
hdss_df['sex'].nunique()

2

In [39]:
import recordlinkage

In [41]:
indexer = recordlinkage.Index()
indexer.block('dob')
pairs = indexer.index(facility_df, hdss_df)

In [42]:
compare = recordlinkage.Compare()

compare.exact('dob', 'dob', label = 'dob')
compare.exact('sex', 'sex', label = 'sex')

compare.string('fullname', 'fullname', threshold = 0.85, label='fullname')
compare.string('petname', 'petname', threshold=0.85, label='petname')

potential_matches=compare.compute(pairs, facility_df, hdss_df)
potential_matches

,,dob,sex,fullname,petname
0,2740,1,1,0.0,0.0
1,2612,1,1,1.0,0.0
3,1178,1,1,1.0,0.0
6,2901,1,1,1.0,1.0
7,2950,1,1,1.0,1.0
...,...,...,...,...,...
2884,2126,1,1,1.0,0.0
2892,3320,1,1,1.0,1.0
2893,691,1,1,1.0,1.0
2898,3446,1,1,1.0,0.0


In [43]:
matches=potential_matches[potential_matches.sum(axis=1)>=3]
matches

,,dob,sex,fullname,petname
1,2612,1,1,1.0,0.0
3,1178,1,1,1.0,0.0
6,2901,1,1,1.0,1.0
7,2950,1,1,1.0,1.0
2409,433,1,1,1.0,1.0
...,...,...,...,...,...
2884,2126,1,1,1.0,0.0
2892,3320,1,1,1.0,1.0
2893,691,1,1,1.0,1.0
2898,3446,1,1,1.0,0.0


In [54]:
duplicate_rows=matches.index.get_level_values(1)
facility_duplicates=facility_df[facility_df.index.isin(duplicate_rows)]
facility_new = facility_df[~facility_df.index.isin(duplicate_rows)]
facility_hdss =hdss_df.append(facility_new)
facility_hdss

<ipython-input-54-3dd5d0163176>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  facility_hdss =hdss_df.append(facility_new)


,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001,Zaina Hanifa,NaN,NaN
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002,Godfrey Maganda,NaN,NaN
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003,Kasim Ngobi,NaN,NaN
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004,Esther,NaN,NaN
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005,Sumaya Swabula,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2895,2897,Swabiru,Aluvin,Najjuma,19-05-2014 00:00,1,N_ID_13670,NaN,NaN,Swabiru Aluvin,4959.0,16-10-2022
2897,2899,Madina,Nakagolo,Nkima,28-07-1955 00:00,2,N_ID_13676,NaN,NaN,Madina Nakagolo,4961.0,17-08-2021
2898,2900,Namulondo,Namugabwe,NaN,14-09-1933 00:00,2,N_ID_13679,NaN,NaN,Namulondo Namugabwe,4962.0,21-06-2019
2899,2901,Ziriya,Kauma,Kato,11-11-1963 00:00,2,N_ID_13682,NaN,NaN,Ziriya Kauma,4963.0,27-12-2018


In [51]:
facility_hdss.duplicated(subset='fullname').sum()

1973